<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

# tpqoa

## Algorithmic Trading with Oanda

`tpqoa` is a wrapper class for the Oanda REST API v20 (http://developer.oanda.com/). It makes use of the Python package `v20` from Oanda (https://github.com/oanda/v20-python). The package is authored and maintained by The Python Quants GmbH. &copy; Dr. Yves J. Hilpisch. MIT License.

## Disclaimer

Trading forex/CFDs on margin carries a high level of risk and may not be suitable for all investors as you could sustain losses in excess of deposits. Leverage can work against you. Due to the certain restrictions imposed by the local law and regulation, German resident retail client(s) could sustain a total loss of deposited funds but are not subject to subsequent payment obligations beyond the deposited funds. Be aware and fully understand all risks associated with the market and trading. Prior to trading any products, carefully consider your financial situation and experience level. Any opinions, news, research, analyses, prices, code examples or other information is provided as general market commentary, and does not constitute investment advice. The Python Quants GmbH will not accept liability for any loss or damage, including without limitation to, any loss of profit, which may arise directly or indirectly from use of or reliance on such information.

The `tpqoa` package is intended as a technological illustration only. It comes with no warranties or representations, to the extent permitted by applicable law.

## Installation

Installing from source via `Git` and `Github`:

    git clone https://github.com/yhilpisch/tpqoa
    cd tpqoa
    python setup.py install
    
Using `pip` in combination with `Github`:

    pip install git+git://github.com/yhilpisch/tpqoa

Using `pip` directly:

    pip install --index-url https://test.pypi.org/simple/ tpqoa

## Connection

In order to connect to the API, you need to have at least a practice account with Oanda (https://oanda.com/). Once logged in to you account, you can create an API token and can copy your account number. These are expected to be stored in a configuration file, with name `oanda.cfg`, for instance, as follows:

    [oanda]
    account_id = XYZ-ABC-...
    access_token = ZYXCAB...
    account_type = practice (default) or live

You can then set up an API connection by instantiating a connection object.

In [1]:
from tpqoa import tpqoa

In [2]:
oanda = tpqoa('oanda.cfg')

## Available Instruments

The `get_instruments()` method retrieves all available instruments.

In [3]:
ins = oanda.get_instruments()

In [4]:
ins[:10]

[('USD/DKK', 'USD_DKK'),
 ('CAD/HKD', 'CAD_HKD'),
 ('AUD/CHF', 'AUD_CHF'),
 ('Silver/CAD', 'XAG_CAD'),
 ('AUD/NZD', 'AUD_NZD'),
 ('USD/CAD', 'USD_CAD'),
 ('AUD/JPY', 'AUD_JPY'),
 ('USD/JPY', 'USD_JPY'),
 ('EUR/TRY', 'EUR_TRY'),
 ('USD/CZK', 'USD_CZK')]

## Historical Data

The `get_history()` method retrieves historical data.

In [5]:
help(oanda.get_history)

Help on method get_history in module tpqoa.tpqoa:

get_history(instrument, start, end, granularity, price) method of tpqoa.tpqoa.tpqoa instance
    Retrieves historical data for instrument.
    
    Parameters
    instrument: string
        valid instrument name
    start, end: datetime, str
        Python datetime or string objects for start and end
    granularity: string
        a string like 'S5', 'M1' or 'D'
    price: string
        one of 'A' (ask) or 'B' (bid)
    
    Returns
    data: pd.DataFrame
        pandas DataFrame object with data



In [6]:
data = oanda.get_history(instrument='EUR_USD',
                  start='2018-01-01',
                  end='2018-08-09',
                  granularity='D',
                  price='A')

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 158 entries, 2018-01-01 22:00:00 to 2018-08-08 21:00:00
Data columns (total 6 columns):
c           158 non-null float64
complete    158 non-null bool
h           158 non-null float64
l           158 non-null float64
o           158 non-null float64
volume      158 non-null int64
dtypes: bool(1), float64(4), int64(1)
memory usage: 7.6 KB


In [8]:
print(data.head())

                           c  complete        h        l        o  volume
time                                                                     
2018-01-01 22:00:00  1.20608      True  1.20822  1.20047  1.20082  110228
2018-01-02 22:00:00  1.20165      True  1.20668  1.20017  1.20620  103380
2018-01-03 22:00:00  1.20710      True  1.20898  1.20050  1.20162  118537
2018-01-04 22:00:00  1.20326      True  1.20846  1.20210  1.20693  112728
2018-01-07 22:00:00  1.19699      True  1.20531  1.19564  1.20310   96212


## Streaming Data

The method `stream_data()` allows the streaming of real-time data (bid & ask).

In [9]:
oanda.stream_data('EUR_USD', stop=3)

2018-12-27T11:54:07.261399703Z 1.13917 1.1393
2018-12-27T11:54:18.934431940Z 1.13914 1.13928
2018-12-27T11:54:19.122899358Z 1.13912 1.13927


By redefining the `on_success()` method, you can control what happes with the streaming data.

In [10]:
class myOanda(tpqoa):
    def on_success(self, time, bid, ask):
        ''' Method called when new data is retrieved. '''
        print('BID: {:.5f} | ASK: {:.5f}'.format(bid, ask))

In [11]:
my_oanda = myOanda('oanda.cfg')

In [12]:
my_oanda.stream_data('EUR_USD', stop=7)

BID: 1.13912 | ASK: 1.13926
BID: 1.13913 | ASK: 1.13927
BID: 1.13913 | ASK: 1.13926
BID: 1.13912 | ASK: 1.13926
BID: 1.13913 | ASK: 1.13927
BID: 1.13912 | ASK: 1.13926
BID: 1.13911 | ASK: 1.13924


## Other Methods

Other major methods are:

In [13]:
help(oanda.create_order)

Help on method create_order in module tpqoa.tpqoa:

create_order(instrument, units) method of tpqoa.tpqoa.tpqoa instance
    Places order with Oanda.
    
    Parameters
    instrument: string
        valid instrument name
    units: int
        number of units of instrument to be bought
        (positive int, eg 'units=50')
        or to be sold (negative int, eg 'units=-100')



In [14]:
help(oanda.get_transactions)

Help on method get_transactions in module tpqoa.tpqoa:

get_transactions(tid=0) method of tpqoa.tpqoa.tpqoa instance
    Retrieves and returns transactions data.



In [15]:
help(oanda.get_account_summary)

Help on method get_account_summary in module tpqoa.tpqoa:

get_account_summary(detailed=False) method of tpqoa.tpqoa.tpqoa instance
    Returns summary data for Oanda account.



<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

<a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:training@tpq.io">training@tpq.io</a>